In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [3]:
df = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/Martin Marietta Pricing Case Study/data/seasonal_imbalanced_shipment_data.csv')
df.head()

,shipment_month,region,product_category,product_id,customer_id,transfer_flag,pickup_or_delivery,Job_distance,Shipment_qty,avg_price_per_ton
0,Mar,Region B,Electronics,P8462,C981783,External Customer,Pickup,10-15,8.31,972.40
1,May,Region D,Chemicals,P2695,C433440,External Customer,Pickup,15-20,16.53,957.58
2,Mar,Region C,Electronics,P7811,C808007,External Customer,Pickup,10-15,12.60,767.09
3,Apr,Region C,Electronics,P1569,C408920,Plant-to-Plant,Delivery,5-10,12.98,951.51
4,Jun,Region A,Chemicals,P1359,C793217,External Customer,Pickup,0-5,38.62,857.31


In [8]:
df['product_category'].value_counts()

,count
product_category,
Food,100334
Electronics,100125
Chemicals,100016
Furniture,99898
Textiles,99627


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500000 entries, 0 to 499999
Data columns (total 10 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   shipment_month      500000 non-null  object 
 1   region              500000 non-null  object 
 2   product_category    500000 non-null  object 
 3   product_id          500000 non-null  object 
 4   customer_id         500000 non-null  object 
 5   transfer_flag       500000 non-null  object 
 6   pickup_or_delivery  500000 non-null  object 
 7   Job_distance        500000 non-null  object 
 8   Shipment_qty        500000 non-null  float64
 9   avg_price_per_ton   500000 non-null  float64
dtypes: float64(2), object(8)
memory usage: 38.1+ MB


Let's find pricing tiers for 3 test cases:

* Electronics being shipped to Region B in Q1
* Textiles being shipped plant-to-plant in Q3
* Furniture delivered in Region C in Dec

In [32]:
segments = {
    'Electronics_Region_B': df.query("product_category == 'Electronics' and region == 'Region B' and shipment_month in ('Jan','Feb','Mar')"),
    'Textiles_Plant_to_Plant': df.query("product_category == 'Textiles'  and shipment_month in ('Jul','Aug','Sep')"),
    'Furniture_Region C': df.query("product_category == 'Furniture' and shipment_month == 'Dec' and region == 'Region C'")
    }

In [33]:
pricing_tiers = {}

for name, group in segments.items():
  band = {
      'floor': group['avg_price_per_ton'].quantile(0.10),
      'middle': group['avg_price_per_ton'].median(),
      'ceiling': group['avg_price_per_ton'].quantile(0.90)
  }

  pricing_tiers[name] = band

segment_pricing = pd.DataFrame(pricing_tiers).T
segment_pricing

,floor,middle,ceiling
Electronics_Region_B,880.552,968.690,1074.618
Textiles_Plant_to_Plant,563.145,667.270,781.207
Furniture_Region C,728.341,799.655,868.239
